LOAD DATASET FOR TB INCIDENCE TO COLAB

In [ ]:
#upload the dataset into google colab
from google.colab import files
uploaded = files.upload()

IMPORT NECESSARY LIBARY

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(10,5), 'figure.dpi':120})


In [ ]:
pf = pd.read_csv('tbcaseskibongoto.csv',header=0,  index_col=0)

In [ ]:
pf['tbcases']

In [ ]:
pf=pf.dropna()

VISUALIZING TB INCIDENCE

In [ ]:
pf.plot(x='Year',title="Kibongoto National TB Hospital TB cases since 2015-2020",ylabel="TB cases")
plt.show()

DATA DECOMPOSE

In [ ]:
decomposition = sm.tsa.seasonal_decompose(pf, model='additive',freq=1)
plt.rcParams["figure.figsize"] = [16,9]
fig = decomposition.plot()

CHECK FOR STATIONARY

In [ ]:
from statsmodels.tsa.stattools import adfuller
adfuller_results=adfuller(pf,autolag='AIC')
print(f'adfuller statistics:{adfuller_results[0]}')
print(f'p-value: {adfuller_results[1]}')
for key,value in adfuller_results[4].items():
  print("Critical value: ")
  print(f'{key},{value}')

FIRST DIFFERENCING

In [ ]:
# 1st Differencing
plt.plot(pf.diff())
plt.show()

In [ ]:
import statsmodels.graphics.tsaplots as sgt
sgt.plot_acf(pf.diff().dropna())
plt.show()


In [ ]:
sgt.plot_pacf(pf.diff().dropna())
plt.show()

In [ ]:
adfuller_results=adfuller(pf.diff().dropna(),autolag='AIC')
print(f'adfuller statistics:{adfuller_results[0]}')
print(f'p-value: {adfuller_results[1]}')
for key,value in adfuller_results[4].items():
  print("Critical value: ")
  print(f'{key},{value}')

In [ ]:
pf.diff()

TEST FOR AUTO-ARIMA

In [ ]:
!pip install --upgrade statsmodels


In [ ]:
pip install pmdarima

In [ ]:
import pmdarima as pm


In [ ]:
auto_arima_fit = pm.auto_arima(pf, start_p=1, start_q=1,
                             max_p=3, max_q=3,  d=1, trace=True,
                             error_action='ignore',
                             suppress_warnings=True,
                             stepwise=True)

SPLITTING TESTING AND TRAINING SET

In [ ]:
# split into train and test sets
X = pf.values
size = int(len(X) * 0.7)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

In [ ]:
history

In [ ]:
test

FIT ARIMA MODEL ARIMA(1,1,0) WITH LOWERST AIC AND BIC

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
# walk-forward validation
for t in range(len(test)):
	model = ARIMA(history, order=(1, 1, 0))
	model_fit = model.fit() 
  
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))

model_fit.summary()

RESIDUAL DIAGNOSIS

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
residuals.plot(label="Residual")

In [ ]:
#residuals.plot(kind='kde')

In [ ]:
#residuals.describe()

PLOT RESULT VS ORIGIONAL

In [ ]:
# plot forecasts against actual outcomes
plt.plot(test)
plt.plot(predictions, color='red')
plt.plot(test,label="Original test TB incidence")
plt.plot(predictions,label="Predicted TB incidence")
plt.legend( loc='upper left')
plt.show()

EVALUATION OF THE MODEL

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error

In [ ]:
rmse =sqrt(mean_squared_error(test, predictions))
print(rmse)

FOECASTED VALUE AND ACTUAL VALUE

In [ ]:
# Train the model on the full dataset 
model = model = ARIMA(pf,order = (1, 1, 0)) 
result = model.fit() 
  
# Forecast for the next 3 years 
forecast = result.predict(start = len(pf),  
                          end = (len(pf)) + 5,  
                          typ = 'levels').rename('Forecast TB cases') 
  
# Plot the forecast values 
pf.plot(figsize = (12, 5), legend = True) 
forecast.plot(legend = True) 
#pf.plot(x='Year',title="TOTAL NEW AND RELAPSE TB INCIDENCE WITH UNKNOWN TREATMENT HISTORY",ylabel="total New and relapse TB cases")
#forecast.plot(legend = True) 
#plt.show()

FORECAST VALUE

In [ ]:
forecast